In [4]:
import requests
from lxml import etree
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager  # 导入 EdgeChromiumDriverManager

import time

# 定义方法
def getWeather(url):
    weather_info = []
    headers = {
        'Cookie': 'UserId=17492113410185356; Hm_lvt_7c50c7060f1f743bccf8c150a646e90a=1749211341; '
                  'HMACCCOUNT=54F2FF78AEECB908; Hm_lvt_5326a74bb3e3143580750a123a85e7a1=1749211410; '
                  'Hm_lpvt_5326a74bb3e3143580750a123a85e7a1=1749212779; '
                  'Hm_lpvt_7c50c7060f1f743bccf8c150a646e90a=1749212779',
       'referer': 'https://lishi.tianqi.com/sanya/201101.html',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                      '(KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'
    }

    # 配置 Edge 浏览器
    edge_options = Options()
    edge_options.add_argument('--headless')  # 无头模式，不显示浏览器窗口
    # 使用 EdgeChromiumDriverManager 自动下载并管理 EdgeDriver
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=edge_options)

    try:
        # 打开网页
        driver.get(url)
        time.sleep(30)  # 延长页面加载等待时间到 10 秒

        # 尝试点击「查看更多」按钮，需要根据实际网页元素修改选择器
        try:
            more_button = driver.find_element(By.XPATH, '/html/body/div[7]/div[1]/div[4]/ul/div')
            more_button.click()
            time.sleep(3)  # 等待更多数据加载
        except Exception as e:
            print(f"未找到查看更多按钮或点击失败: {e}")

        # 获取页面源代码
        # ... 已有代码 ...

        # 获取页面源代码
        page_source = driver.page_source

        # 数据预处理
        resp_html = etree.HTML(page_source)

        # 修正xpath表达式
        resp_list = resp_html.xpath("//ul[@class='thrui']/li")

        for li in resp_list:
            day_weather_info = {}

            # 日期
            date_text = li.xpath("./div[1]/text()")
            day_weather_info['date_time'] = date_text[0].split(' ')[0] if date_text else ""

            # 最高气温（处理摄氏度符号）
            high_text = li.xpath("./div[2]/text()")
            high = high_text[0] if high_text else ""
            day_weather_info['high'] = high.replace("°C", "")

            # 最低气温
            low_text = li.xpath("./div[3]/text()")
            low = low_text[0] if low_text else ""
            day_weather_info['low'] = low.replace("°C", "")

            # 天气
            weather_text = li.xpath("./div[4]/text()")
            day_weather_info['weather'] = weather_text[0] if weather_text else ""

            weather_info.append(day_weather_info)

    except Exception as e:
        print(f"获取{url}数据时出错: {e}")
    finally:
        driver.quit()  # 关闭浏览器

    return weather_info


weathers = []
for month in range(1, 13):
    weather_time = '2011' + ('0' + str(month) if month < 10 else str(month))
    print(f"正在获取{weather_time}的天气数据...")
    url = f'https://lishi.tianqi.com/sanya/{weather_time}.html'
    weather = getWeather(url)

    # 存储数据
    weathers.extend(weather)  # 使用extend而不是append，避免嵌套列表

    # 打印进度
    print(f"{weather_time}获取完成, 共{len(weather)}条记录")

# 数据写入CSV
try:
    with open('weather.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile)
        # 写入列名
        writer.writerow(['日期', '最高气温', '最低气温', '天气'])

        # 写入数据
        for day_data in weathers:
            writer.writerow([
                day_data['date_time'],
                day_data['high'],
                day_data['low'],
                day_data['weather']
            ])

    print(f"数据已成功写入 weather.csv, 共{len(weathers)}条记录")

except Exception as e:
    print(f"写入CSV文件时出错: {e}")

正在获取201101的天气数据...
201101获取完成, 共20条记录
正在获取201102的天气数据...
201102获取完成, 共28条记录
正在获取201103的天气数据...
201103获取完成, 共31条记录
正在获取201104的天气数据...
201104获取完成, 共30条记录
正在获取201105的天气数据...
201105获取完成, 共10条记录
正在获取201106的天气数据...
201106获取完成, 共30条记录
正在获取201107的天气数据...
201107获取完成, 共10条记录
正在获取201108的天气数据...
201108获取完成, 共31条记录
正在获取201109的天气数据...
201109获取完成, 共30条记录
正在获取201110的天气数据...
201110获取完成, 共31条记录
正在获取201111的天气数据...
201111获取完成, 共30条记录
正在获取201112的天气数据...
201112获取完成, 共31条记录
数据已成功写入 weather.csv, 共312条记录


In [5]:
import pandas as pd
weather=pd.read_csv('weather.csv' ,encoding='utf-8')

In [6]:
weather

,日期,最高气温,最低气温,天气
0,2011-01-01,27℃,18℃,多云
1,2011-01-02,27℃,18℃,多云
2,2011-01-03,28℃,19℃,多云
3,2011-01-04,28℃,19℃,多云
4,2011-01-05,29℃,18℃,多云
...,...,...,...,...
307,2011-12-27,24℃,19℃,多云
308,2011-12-28,25℃,21℃,多云
309,2011-12-29,27℃,21℃,晴~多云
310,2011-12-30,27℃,21℃,多云
